In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import itertools
from tqdm import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, roc_curve, auc, confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import (
    Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, 
    Conv1D ,GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,
    Conv2D, MaxPool2D, concatenate,
    Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D, Bidirectional, 
)
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import metrics
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import backend as K

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
DEBUG = False
EMBED_SIZE = 300
MAX_FEATURES = 10000 if DEBUG else 90000
SEQUENCE_LENGTH = 50
GLOVE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
WIKI = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
PARAGRAM = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'

 # Load  data

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
if DEBUG:
    train_df = train_df[:100000]
train_df['question_text'] = train_df['question_text'].str.lower()
test_df['question_text'] = test_df['question_text'].str.lower()
DEBUG, train_df.shape, test_df.shape

# Embedding

In [ ]:
def load_embedding(file, tokenizer):
    def split(line):
        word, *arr = line.split(' ')
        return word, np.asarray(arr, dtype='float32')
    with open(file, encoding='utf8', errors='ignore') as f:
        if DEBUG:
            embeddings = dict(split(line) for line in tqdm(itertools.islice(f, 10000)) if len(line) > 100)
        else:
            embeddings = dict(split(line) for line in tqdm(f) if len(line) > 100)
    values = np.stack(embeddings.values())
    mean = values.mean()
    std = values.std()
    n_words = min(MAX_FEATURES, len(tokenizer.word_index))
    
    embedding_matrix = np.random.normal(mean, std, (n_words, EMBED_SIZE))
    for word, i in tokenizer.word_index.items():
        if i < MAX_FEATURES and word in embeddings:
            embedding_matrix[i] = embeddings[word]
            
    return embedding_matrix

# Models

In [ ]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b
        eij = K.tanh(eij)
        a = K.exp(eij)
        
        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
def model_gru_atten_3(embedding):
    input_ = Input(shape=(SEQUENCE_LENGTH,))
    x = Embedding(MAX_FEATURES, EMBED_SIZE, weights=[embedding])(input_)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(96, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Attention(SEQUENCE_LENGTH)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
class AveragedModel:
    def __init__(self, tokenizer):
        self.embeddings = [
            load_embedding(GLOVE, tokenizer),
            load_embedding(WIKI, tokenizer),
            load_embedding(PARAGRAM, tokenizer),
        ]
        self.models = [model_gru_atten_3(embedding) for embedding in self.embeddings]
        
    def fit(self, *args, **kwargs):
        for model in self.models:
            model.fit(*args, **kwargs)
            
    def predict(self, *args, **kwargs):
        outputs = [model.predict(*args, **kwargs) for model in self.models]
        return np.array(outputs).mean(axis=0)

# Cross Validation

In [ ]:
# X = train_df["question_text"]
# y = train_df['target'].values
# predicted_y = np.zeros_like(y, dtype='float32')

# indices = []
# splitted_Xy = []
# models = []

# splitter = StratifiedKFold(2, random_state=0)


# for train_index, test_index in splitter.split(X, y):
#     indices.append((train_index, test_index))
    
#     tokenizer = Tokenizer(num_words=MAX_FEATURES)
#     tokenizer.fit_on_texts(X[train_index])
    
#     train_X = tokenizer.texts_to_sequences(X[train_index])
#     train_X = pad_sequences(train_X, maxlen=SEQUENCE_LENGTH)
#     test_X = tokenizer.texts_to_sequences(X[test_index])
#     test_X = pad_sequences(test_X, maxlen=SEQUENCE_LENGTH)
#     train_y = y[train_index]
#     test_y = y[test_index]

#     splitted_Xy.append((train_X, test_X, train_y, test_y))
#     model = AveragedModel(tokenizer)
#     models.append(model)

In [ ]:
# for model, (train_X, test_X, train_y, test_y), (train_index, test_index) in zip(models, splitted_Xy, indices):
#     model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(test_X, test_y), verbose=0)
    
#     yp = model.predict([test_X], batch_size=512, verbose=0).flatten()
#     predicted_y[test_index] = yp

In [ ]:
# def plot_base():
#     plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver operating characteristic example')
#     plt.legend(loc="lower right")
#     plt.show()

In [ ]:
# fpr, tpr, thresholds = roc_curve(y, predicted_y, pos_label=1)
# roc_auc = auc(fpr, tpr)
# plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % roc_auc)
    
# plot_base()

In [ ]:
# threshold = 0.5
# confusion_matrix(y, (predicted_y > threshold).astype(int))

In [ ]:
# f1_score(y, predicted_y > threshold)

In [ ]:
# Save for later plotting
# def find_best_threshold(y, predicted_y):
#     best_f1, best_threshold = 0, 0
#     for threshold in np.arange(0.01, 0.99, 0.01):
#         f1 = f1_score(y, predicted_y > threshold)
#         if f1 > best_f1:
#             best_f1, best_threshold = f1, threshold
#     return best_f1, best_threshold

# best_f1, best_threshold = find_best_threshold(y, predicted_y)
# best_f1, best_threshold

# Train and predict

In [ ]:
# tokenizer = Tokenizer(num_words=MAX_FEATURES)
# tokenizer.fit_on_texts(X)

# train_X = tokenizer.texts_to_sequences(X)
# train_X = pad_sequences(train_X, maxlen=SEQUENCE_LENGTH)
# train_y = y

# model = AveragedModel(tokenizer)
# model.fit(train_X, train_y, batch_size=512, epochs=1, verbose=0)

In [ ]:
# test_X = tokenizer.texts_to_sequences(test_df["question_text"])
# test_X = pad_sequences(test_X, maxlen=SEQUENCE_LENGTH)

# print('Tokenized test dataset')
# prediction = model.predict([test_X], batch_size=1024, verbose=0).flatten()
# predicted_label = (prediction > threshold).astype(int)
# positive = predicted_label.mean()
# print(f'Tested on {model}: {positive:.3%} positive')

In [ ]:
# out_df = pd.read_csv('../input/test.csv', usecols=['qid'])
# out_df = pd.DataFrame({'qid': out_df['qid'].values})
# out_df['prediction'] = predicted_label
# out_df.to_csv("submission.csv", index=False)

# Robustness Validation

In [ ]:
# import random
# def drop_randomly(sequences, ratio=.1):
#     for sequence in sequences:
#         if random.random() < ratio and sequence:
#             del sequence[random.randint(0, len(sequence) - 1)]

In [ ]:
# X = train_df["question_text"]
# y = train_df['target'].values
# predicted_y = np.zeros_like(y, dtype='float32')

# indices = []
# splitted_Xy = []
# models = []

# splitter = StratifiedKFold(2, random_state=0)


# for train_index, test_index in splitter.split(X, y):
#     indices.append((train_index, test_index))
    
#     tokenizer = Tokenizer(num_words=MAX_FEATURES)
#     tokenizer.fit_on_texts(X[train_index])
    
#     train_X = tokenizer.texts_to_sequences(X[train_index])
#     drop_randomly(train_X, ratio=.1) # test for robustness
#     train_X = pad_sequences(train_X, maxlen=SEQUENCE_LENGTH)
#     test_X = tokenizer.texts_to_sequences(X[test_index])
#     test_X = pad_sequences(test_X, maxlen=SEQUENCE_LENGTH)
#     train_y = y[train_index]
#     test_y = y[test_index]

#     splitted_Xy.append((train_X, test_X, train_y, test_y))
#     model = AveragedModel(tokenizer)
#     models.append(model)

In [ ]:
# for model, (train_X, test_X, train_y, test_y), (train_index, test_index) in zip(models, splitted_Xy, indices):
#     model.fit(train_X, train_y, batch_size=512, epochs=1, verbose=0)
    
#     yp = model.predict([test_X], batch_size=1024, verbose=0).flatten()
#     predicted_y[test_index] = yp
    
    
# fpr, tpr, thresholds = roc_curve(y, predicted_y, pos_label=1)
# roc_auc = auc(fpr, tpr)
# plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % roc_auc)
# plot_base()

# Refinement

In [ ]:
# rocs = [(fpr, tpr, thresholds, roc_auc, best_f1, best_threshold)]
# for i in range(2):
#     for model, (train_X, test_X, train_y, test_y), (train_index, test_index) in zip(models, splitted_Xy, indices):
#         model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(test_X, test_y), verbose=0)
#         yp = model.predict([test_X], batch_size=1024, verbose=0).flatten()
#         predicted_y[test_index] = yp
        
#     fpr_, tpr_, thresholds_ = roc_curve(y, predicted_y, pos_label=1)
#     roc_auc_ = auc(fpr_, tpr_)
    
#     best_f1, best_threshold = find_best_threshold(y, predicted_y)
#     rocs.append((fpr_, tpr_, thresholds_, roc_auc_, best_f1, best_threshold))
#     print(f'Epoch {i + 2}: ROC_AUC={roc_auc_}')
#     print('Confusion matrix', confusion_matrix(y, (predicted_y > best_threshold).astype(int)))
    
# rocs

In [ ]:
# plt.figure(figsize=(10, 7))
# for i, (fpr_, tpr_, thresholds_, roc_auc_, best_f1, best_threshold) in enumerate(rocs):
#     plt.plot(fpr_, tpr_, label=f'{i + 1} epochs (area = {roc_auc_:.3f}, best f1={best_f1:.3f})')
# plot_base()

In [ ]:
# aucs = [i[3] for i in rocs]
# argmax = np.argmax(aucs)
# best_epochs = argmax + 1
# best_threshold = rocs[argmax][5]
# aucs, best_epochs, aucs[argmax], best_threshold

In [ ]:
best_epochs = 2
best_threshold = 0.35
X = train_df["question_text"]
y = train_df['target'].values

tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(X)

train_X = tokenizer.texts_to_sequences(X)
train_X = pad_sequences(train_X, maxlen=SEQUENCE_LENGTH)
train_y = y

model = AveragedModel(tokenizer)
model.fit(train_X, train_y, batch_size=512, epochs=best_epochs, verbose=0)


test_X = tokenizer.texts_to_sequences(test_df["question_text"])
test_X = pad_sequences(test_X, maxlen=SEQUENCE_LENGTH)

print('Tokenized test dataset')
prediction = model.predict([test_X], batch_size=1024, verbose=0).flatten()
predicted_label = (prediction > best_threshold).astype(int)
positive = predicted_label.mean()
print(f'Tested on {model}: {positive:.3%} positive')

out_df = pd.read_csv('../input/test.csv', usecols=['qid'])
out_df = pd.DataFrame({'qid': out_df['qid'].values})
out_df['prediction'] = predicted_label
out_df.to_csv("submission.csv", index=False)

In [ ]:
def visualize_layers():
    plt.figure(figsize=(20, 8))
    for i, sub_model in enumerate(model.models):
        for j in range(3):
            plt.subplot(3, 3, i * 3 + j + 1)
            layer = sub_model.layers[j + 2]
            weights = layer.backward_layer.get_weights()[1]
            plt.imshow(weights, cmap='viridis', interpolation='nearest')
            plt.title(f'Embedding {i + 1}, bidirection GRU layer {j + 1}')
visualize_layers()